### Menyiapkan Source Dataset
Source Dataset akan digunakan untuk melatih CNN sebelum dilakukan proses Transfer Learning

In [37]:
import pandas as pd
import numpy as np

In [38]:
data_path = "data/data_source/"
filename = "islamicQA.csv"

In [39]:
train_df = pd.read_csv(data_path + filename, delimiter=',')
train_df.head()

,Kategori,Pertanyaan,Jawaban
0,adab akhlak dan pensucian jiwa,terkait dengan penebus ghibah apakah cukup de...,alhamdulillah ghibah merupakan dosa...
1,adab akhlak dan pensucian jiwa,sampai kapan allah memaafkan hambanya yang ber...,alhamdulillah allah ta ala berfirman ...
2,adab akhlak dan pensucian jiwa,aku mengajukan hutang kepada salah satu bank s...,alhamdulillah pertama diharamka...
3,adab akhlak dan pensucian jiwa,mengapa para shahabat yang dijamin masuk surga...,alhamdulillah pertama tidak diragukan...
4,adab akhlak dan pensucian jiwa,apa tanda cinta allah kepada seorang hamba da...,alhamdulillahsungguh anda telah bertanya denga...


In [40]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (kategori_total_list)

fiqih dan usul fiqih                1043
akidah                               234
fiqih keluarga                       206
hadis dan ilmu ilmunya                64
adab  akhlak dan pensucian jiwa       64
al qur an dan ilmu al qur an          50
ilmu dan dakwah                       34
politik islam                         15
sejarah dan biografi                  14
pendidikan                            13
problematika kejiwaan dan sosial       6
Name: Kategori, dtype: int64


In [41]:
kategori = train_df["Kategori"].values
tanya = train_df["Pertanyaan"].values
jawab = train_df["Jawaban"].values

In [42]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data
    
def myTokenizer(content, lower=True):
    raw = content.split(' ')
    remover = re.compile("[^a-zA-Z-]")
    
    token = []
    
    for i in raw:
        term = remover.sub('', i)
        if lower == True:
            term = term.lower()
        token.append(term)
#     tokenized = filter(None, token)
    
    return token

def sentenceToVec(string):
    if type(string) is not str:
        return np.zeros((300,), dtype=float)
    
    string = string.replace('\n', '')
    string = np.array(myTokenizer(string))
    
    feature = None
    begin = True
    for word in string:
        stat, vec = getWordEmbedding(word, cursor)
        if not stat:
            continue
        if begin:
            begin = False
            feature = vec
        else:
            feature += vec
            # feature = np.concatenate([feature, vec])
    
    if feature is not None:
        feature = feature/np.linalg.norm(feature)
    else:
        feature = np.zeros((300,), dtype=float)
    
    return feature

import re
import json
import unicodedata
import mysql.connector
from tqdm import tqdm

db = mysql.connector.connect(user="root", password='', database="glove")
cursor = db.cursor(buffered=True)

pbar = tqdm(total=len(tanya))
ftr = []
for i in tanya:
    try:
        ftr.append(sentenceToVec(i))
    except Exception as err:
        print (err)
        print (i)
        break
    pbar.update(1)
pbar.close()
ftr = np.array(ftr)

100%|██████████████████████████████████████████████████████████████████████████████| 1743/1743 [04:39<00:00,  6.23it/s]


In [43]:
ftr

array([[ 0.06532823,  0.05884961, -0.01162243, ...,  0.02337618,
         0.07418328,  0.02370651],
       [ 0.05333917,  0.03406325, -0.047201  , ...,  0.06848152,
         0.02441938, -0.0102491 ],
       [ 0.05779844,  0.02963271, -0.04286807, ...,  0.06795678,
         0.06174846, -0.00555836],
       ...,
       [-0.0167215 ,  0.04160699, -0.03811455, ...,  0.05412887,
         0.03542367,  0.02206324],
       [ 0.05239877,  0.00257464, -0.05256515, ...,  0.05086377,
         0.0189409 ,  0.03856589],
       [ 0.06875055,  0.00971227, -0.05302512, ...,  0.06848325,
         0.05816825,  0.03025898]], dtype=float32)

In [44]:
nan_idx = []
for i in range(len(kategori)):
    if kategori[i] is np.nan:
        nan_idx.append(i)
kategori_c = np.delete(kategori, nan_idx)
ftr_c = np.delete(ftr, nan_idx, 0)

In [45]:
ftr_c

array([[ 0.06532823,  0.05884961, -0.01162243, ...,  0.02337618,
         0.07418328,  0.02370651],
       [ 0.05333917,  0.03406325, -0.047201  , ...,  0.06848152,
         0.02441938, -0.0102491 ],
       [ 0.05779844,  0.02963271, -0.04286807, ...,  0.06795678,
         0.06174846, -0.00555836],
       ...,
       [-0.0167215 ,  0.04160699, -0.03811455, ...,  0.05412887,
         0.03542367,  0.02206324],
       [ 0.05239877,  0.00257464, -0.05256515, ...,  0.05086377,
         0.0189409 ,  0.03856589],
       [ 0.06875055,  0.00971227, -0.05302512, ...,  0.06848325,
         0.05816825,  0.03025898]], dtype=float32)

In [46]:
cls = []
kategori_u = pd.unique(kategori_c)
kategori_u = kategori_u.tolist()
for i in range(len(kategori_c)):
    one_hot = np.zeros((len(kategori_u),), dtype=int)
    idx = kategori_u.index(kategori_c[i])
    one_hot[idx] = 1
    cls.append(one_hot)
cls = np.array(cls)

print ("Mengecek array size dari fitur dan kelas...")
print ("size of ftr " + str(ftr_c.shape))
print ("size of cls " + str(cls.shape))

Mengecek array size dari fitur dan kelas...
size of ftr (1743, 300)
size of cls (1743, 11)


In [47]:
np.save(data_path + "ftr-data-source.npy", ftr_c)
np.save(data_path + "cls-data-source.npy", cls)

## SMOTE
Source Dataset ternyata tidak imbang antara satu kelas dan kelas lainnya. Maka, digunakan algoritma **SMOTE** untuk membuat jumlah data dan setiap kelasnya imbang (*balanced*).

In [48]:
ftr = np.load(data_path + "ftr-data-source.npy")
cls = np.load(data_path + "cls-data-source.npy")

In [49]:
indexed_cls = list(map(lambda x: np.argmax(x), cls))
print(pd.unique(indexed_cls))

[ 0  1  2  3  4  5  6  7  8  9 10]


In [50]:
def valuesCount(arr):
    cls_dict = {}
    for i in arr:
        if i in cls_dict:
            cls_dict[i] += 1
        else:
            cls_dict[i] = 1
    return cls_dict

In [51]:
valuesCount(indexed_cls)

{0: 64,
 1: 234,
 2: 50,
 3: 1043,
 4: 206,
 5: 64,
 6: 34,
 7: 13,
 8: 15,
 9: 6,
 10: 14}

In [52]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
ftr_res, cls_res = sm.fit_sample(ftr, indexed_cls)

In [53]:
valuesCount(cls_res)

{0: 1043,
 1: 1043,
 2: 1043,
 3: 1043,
 4: 1043,
 5: 1043,
 6: 1043,
 7: 1043,
 8: 1043,
 9: 1043,
 10: 1043}

In [54]:
def convToOneHot(arr):
    total_cls = len(pd.unique(arr))
    cls = []
    for i in arr:
        temp_vec = np.zeros((total_cls,), dtype=int)
        temp_vec[i] = 1
        cls.append(temp_vec)
    cls = np.array(cls)
    return cls

In [55]:
cls_fin = convToOneHot(cls_res)
print ("Panjang array kelas seletah convToOneHot: " + str(len(cls_fin[0])))

Panjang array kelas seletah convToOneHot: 11


In [56]:
np.save(data_path + "ftr_data_source_balanced.npy", ftr_res)
np.save(data_path + "cls_data_source_balanced.npy", cls_fin)

## Menyiapkan Target Dataset
Menyiapkan Target Dataset, mengubah bentuknya agar bisa menjadi masukan untuk CNN.

In [57]:
import re
import json
import unicodedata
import mysql.connector
import numpy as np
import pandas as pd

In [58]:
data_target_path = "data/data_target/"
filename_data_target = "questions2.csv"

In [59]:
def arrangeInputMatrix(data, labels):
    ftr_list = []
    cls_list = []
    
    unique_labels = pd.unique(labels)
    total_labels = len(unique_labels)
    
    for i in range(len(data)):
        cls = np.zeros((total_labels,), dtype=int)
        cls_idx = np.where(unique_labels == labels[i])[0][0]
        cls[cls_idx] = 1
        
        string = data[i].replace('\n', '')
        string = np.array(myTokenizer(string))

        begin = True
        for word in string:
            stat, vec = getWordEmbedding(word, cursor)
            if not stat:
                continue
            if begin:
                begin = False
                feature = vec
            else:
                feature += vec
                # feature = np.concatenate([feature, vec])

        feature = feature/np.linalg.norm(feature)
        ftr_list.append(feature)
        cls_list.append(cls)
        
    return np.array(ftr_list), np.array(cls_list)

In [60]:
print ("convert into numpy array")
df = pd.read_csv(data_target_path + filename_data_target, sep=',', names=["pertanyaan", "kategori"])
tanya_target = df["pertanyaan"].values
label_target = df["kategori"].values

convert into numpy array


In [61]:
ftr, cls = arrangeInputMatrix(tanya_target, label_target)

In [62]:
print ("checking len of array")
print (len(tanya_target) == len(ftr))
print (len(tanya_target) == len(cls))

print ("checking size of arrays")
print ("size of ftr: " + str(ftr.shape))
print ("size pf cls: " + str(cls.shape))

checking len of array
True
True
checking size of arrays
size of ftr: (452, 300)
size pf cls: (452, 72)


In [63]:
np.save(data_target_path + "ftr_data_target.npy", ftr)
np.save(data_target_path + "cls_data_target.npy", cls)

## Splitting Dataset
Memisahkan Source Dataset dan Target Dataset menjadi Train Set dan Test Tes

## Splitting Target Dataset

In [64]:
ftr_data_target = np.load(data_target_path + "ftr_data_target.npy")
cls_data_target = np.load(data_target_path + "cls_data_target.npy")

In [65]:
print (ftr_data_target.shape)
print(cls_data_target.shape)

(452, 300)
(452, 72)


In [66]:
data_target_path_splitted = "data/splitted/data_target/"

In [67]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=0)
# for train_index, test_index in sss.split(ftr_data_target, cls_data_target):
#     X, X_test = ftr_data_target[train_index], ftr_data_target[test_index]
#     Y, Y_test = cls_data_target[train_index], cls_data_target[test_index]
#     break

X, X_test, Y, Y_test = train_test_split(ftr_data_target, cls_data_target, test_size=0.1, random_state=42) #stratify agar pembagian tiap kelas merata

In [68]:
np.save(data_target_path_splitted + "X.npy", X)
np.save(data_target_path_splitted + "Y.npy", Y)
np.save(data_target_path_splitted + "X_test.npy", X_test)
np.save(data_target_path_splitted + "Y_test.npy", Y_test)

## Splitting Source Dataset

In [69]:
data_source_path = "data/data_source/"

ftr_data_source = np.load(data_source_path + "ftr_data_source_balanced.npy")
cls_data_source = np.load(data_source_path + "cls_data_source_balanced.npy")

In [70]:
data_source_path_splitted = "data/splitted/data_source/"

In [71]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(ftr_data_source, cls_data_source):
    X, X_test = ftr_data_source[train_index], ftr_data_source[test_index]
    Y, Y_test = cls_data_source[train_index], cls_data_source[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

In [72]:
np.save(data_source_path_splitted + "X.npy", X)
np.save(data_source_path_splitted + "Y.npy", Y)
np.save(data_source_path_splitted + "X_test.npy", X_test)
np.save(data_source_path_splitted + "Y_test.npy", Y_test)

# coba pake RF

In [24]:
import numpy as np
# x = np.load("data/data_target/ftr_data_target.npy")
# y = np.load("data/data_target/cls_data_target.npy")
# X_train = np.load("data/splitted/data_target/X.npy")
# y_train = np.load("data/splitted/data_target/Y.npy")
# X_test = np.load("data/splitted/data_target/X_test.npy")
# y_test = np.load("data/splitted/data_target/Y_test.npy")
X_train = np.load("data/splitted/data_source/X.npy")
y_train = np.load("data/splitted/data_source/Y.npy")
X_test = np.load("data/splitted/data_source/X_test.npy")
y_test = np.load("data/splitted/data_source/Y_test.npy")

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
RandomForest = RandomForestClassifier()
RandomForest.fit(X_train, y_train)
y_RF = RandomForest.predict(X_test)
print(accuracy_score(y_test, y_RF))
print(precision_recall_fscore_support(y_test, y_RF, average='weighted'))

c:\users\purina qa\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.877177700348432
(0.9978996952737074, 0.877177700348432, 0.9131293941173817, None)


In [30]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# classifier = svm.SVC(kernel='linear', C=0.01)
# y_pred = classifier.fit(X_train, y_train).predict(X_test)

for kernel in tqdm(('linear', 'poly', 'rbf')):
    dSVM = svm.SVC(kernel=kernel, gamma='auto')
    dSVM.fit(X_train, y_train)
    y_SVM = dSVM.predict(X_test)
    print(accuracy_score(y_test, y_SVM))
    print(precision_recall_fscore_support(y_test, y_SVM, average='micro'))

  0%|                                                       | 0/3 [00:00<?, ?it/s]


ValueError: bad input shape (10325, 11)

In [27]:
#Tanpa CrossVal sebelum Oversampling
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.5)
nbc = mnb.fit(X_train, y_train) # Kelemahan Implementasinya disini
y_nbc = nbc.predict(X_test)
print(accuracy_score(y_test, y_nbc))
print(precision_recall_fscore_support(y_test, y_nbc, average='weighted'))

ValueError: bad input shape (10325, 11)